In [ ]:
# Load libraries
using Plots, Rasters, ArchGDAL 
using WhereTheWaterFlows, ImageComponentAnalysis
const WWF = WhereTheWaterFlows # this is how module aliasing is done in Julia

In [ ]:
function PreProcessTopo( DEM )
    h    = Float32.(DEM.data[:,:,1])::Matrix{Float32}
    h   .= h[:,end:-1:1]
    h[h.==DEM.missingval] .= NaN
    return h
end

function PreProcessCoords( DEM_array )
    x     = Array(DEM_array)
    x   .-= x[1]        
    x   .*= 111e3
end

#### Step 0: Read the data

In [ ]:
DEM              = Raster("../data/MontBlanc.tif")

In [ ]:
h     = PreProcessTopo( DEM )                     # read h and 
x     = PreProcessCoords( DEM.dims[1] )           # read x
y     = PreProcessCoords( DEM.dims[2] )[end:-1:1] # read y and reverse it
h_rev = .-copy(h);                                # flip the topography

#### Step 1: Plot the data

In [ ]:
p = heatmap(x./1e3, y./1e3, h', color=:terrain, aspect_ratio=1, 
xlabel="x [km]", ylabel="y [km]", title="DEM nearby Mt Blanc - Altitude [m]")

In [ ]:
p = heatmap(x./1e3, y./1e3, h_rev', color=:terrain, aspect_ratio=1, 
xlabel="x [km]", ylabel="y [km]", title="Reversed DEM - Altitude [m]")

#### Step 2: Let's fill the sinks using WhereTheWaterFlows

In [ ]:
wf = waterflows(h)                                                    # Once on h                  
area1, slen1, dir1, nout1, nin1, pits1, c1, bnds1 = waterflows(h_rev) # Once on h_rev
h_rev_filled = fill_dem(h_rev, pits1, dir1);                          # Fill the reversed topo

p = heatmap(x./1e3, y./1e3, h_rev_filled', color=:terrain, aspect_ratio=1, 
xlabel="x [km]", ylabel="y [km]", title="Filled reversed DEM - Altitude [m]")

#### Step 3: Difference between the filled reversed DEM and the reversed DEM

In [ ]:
summits = h_rev_filled .- h_rev;

p = heatmap(x./1e3, y./1e3, summits', color=:terrain, aspect_ratio=1, 
xlabel="x [km]", ylabel="y [km]", title="Summits - Altitude [m]")

#### Step 4: Mask the summits

In [ ]:
ϵ = 50.0
mask_summits = summits .> ϵ # filter noise au passage

p = heatmap(x./1e3, y./1e3, mask_summits', color=:lajolla, aspect_ratio=1, 
xlabel="x [km]", ylabel="y [km]", title="Summits - Altitude [m]")

#### Step 5: Label components using ImageComponentAnalysis

In [ ]:
components = label_components(mask_summits)

p = heatmap(x./1e3, y./1e3, components', color=:turbo, aspect_ratio=1, 
xlabel="x [km]", ylabel="y [km]", title="Summits - Components (n = $(maximum(components)) with ϵ = $(ϵ) m)")